<a href="https://colab.research.google.com/github/Thiago-Reis-Porto/treinamento-h2ia/blob/main/Relat%C3%B3rio_Buscas_sem_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)


# Resolver o quebra-cabeças usando Buscas

In [3]:
import numpy as np

solution = np.array([[1,2,3],[4,5,6],[7,8,0]])

#---------------------------------------------------------------------------------------------------------
#--Classe do quebra-cabeça--------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------

class Puzzle:
  
  #Construtor recebe o pai e o estado(ou gera um estado aleatorio)
  def __init__(self, parent, state = np.random.choice([i for i in range(9)] , size=(3, 3), replace=False)):
    self.state = state #array com o estado
    self.blankPos = list(map(lambda x: x[0],(np.where(self.state == 0))) ) #posição do espaço
    self.isSolution = np.array_equal(solution,self.state) 
    self.visited = False
    self.children = [] #proximos movimentos possiveis
    self.parent = parent

#---------------------------------------------------------------------------------------------------------
#-Movimentos - -------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------
  #Move espaço para esquerda se possivel
  def __moveL(self):
    blankPos = self.blankPos
    if blankPos[1] == 0:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]][blankPos[1]-1]
    newState[blankPos[0]][blankPos[1]-1] = 0
    return newState
  
  #Move espaço para direita se possivel
  def __moveR(self):
    blankPos = self.blankPos
    if blankPos[1] == 2:
      return np.array([])
    newState = self.state.copy()    
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]][blankPos[1]+1]
    newState[blankPos[0]][blankPos[1]+1] = 0
    return newState
  
  #Move espaço para cima se possivel
  def __moveU(self):
    blankPos = self.blankPos
    if blankPos[0] == 0:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]-1][blankPos[1]]
    newState[blankPos[0]-1][blankPos[1]] = 0
    return newState
  #Move espaço para baixo se possivel
  def __moveD(self):
    blankPos = self.blankPos
    if blankPos[0] == 2:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]+1][blankPos[1]]
    newState[blankPos[0]+1][blankPos[1]] = 0
    return newState

#---------------------------------------------------------------------------------------------------------
#-----------Gera os proximos movimentos possiveis a partir do estado atual--------------------------------
#---------------------------------------------------------------------------------------------------------
  def genChildren(self):
    moves = [self.__moveR, self.__moveL, self.__moveU, self.__moveD]
    for i in moves:
      newMove = i()
      if newMove.any() and self.parent == None:
        self.children.append(Puzzle(self, newMove))
      
      elif newMove.any() and not np.array_equal(newMove, self.parent.state): 
        self.children.append(Puzzle(self, newMove))



## Busca em largura

In [22]:
from collections import deque

def bfs(puzzle):
  if puzzle.isSolution:
    return puzzle
  
  q = deque()
  q.append(puzzle)

  while (True):
    q[0].genChildren()
    for i in q[0].children:
      if i.isSolution:
        return i
      q.append(i)
    q.popleft()


In [23]:
b =np.array([[1, 3, 4],[8, 7, 2],[6, 0, 5]])
p = Puzzle(None,state=b)
s = bfs(p)
pr = s
#-----printa solução------------
while (pr.parent):
  print(str(pr.state))
  print("\n")
  pr=pr.parent
print(p.state)

[[1 2 3]
 [4 5 6]
 [7 8 0]]


[[1 2 3]
 [4 5 0]
 [7 8 6]]


[[1 2 3]
 [4 0 5]
 [7 8 6]]


[[1 2 3]
 [0 4 5]
 [7 8 6]]


[[1 2 3]
 [7 4 5]
 [0 8 6]]


[[1 2 3]
 [7 4 5]
 [8 0 6]]


[[1 2 3]
 [7 4 5]
 [8 6 0]]


[[1 2 3]
 [7 4 0]
 [8 6 5]]


[[1 2 3]
 [7 0 4]
 [8 6 5]]


[[1 0 3]
 [7 2 4]
 [8 6 5]]


[[1 3 0]
 [7 2 4]
 [8 6 5]]


[[1 3 4]
 [7 2 0]
 [8 6 5]]


[[1 3 4]
 [7 0 2]
 [8 6 5]]


[[1 3 4]
 [0 7 2]
 [8 6 5]]


[[1 3 4]
 [8 7 2]
 [0 6 5]]


[[1 3 4]
 [8 7 2]
 [6 0 5]]


## Busca em Profundidade

In [17]:
def dfs(puzzle):
  def isVisited(ar, vst):
    for i in vst:
      if np.array_equal(ar, i):
        return True
    return False

  if puzzle.isSolution:
    return puzzle
  
  q = deque()
  q.append(puzzle)
  visited = []

  while (True):
    vstd = isVisited(q[-1].state, visited)
    
    if vstd and not q[-1].children:
      q.pop()
      q[-1].children.pop(0)
      continue
    
    if not vstd:
      visited.append(q[-1].state)
      q[-1].genChildren() 
    
    if q[-1].children[0].isSolution:
      return q[-1].children[0]
    q.append(q[-1].children[0])


In [32]:
b = np.array([[1, 2, 3],[4, 5, 6],[0, 7, 8]])
p = Puzzle(None,b)
pr = dfs(p)
pr = s
#-----printa solução------------
while (pr.parent):
  print(str(pr.state))
  print("\n")
  pr=pr.parent
print(p.state)

[[1 2 3]
 [4 5 6]
 [7 8 0]]


[[1 2 3]
 [4 5 6]
 [7 0 8]]


[[1 2 3]
 [4 5 6]
 [0 7 8]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



1.

**BFS**: Possui maior custo de memoria a fila, dos nós que deve ser mantida para processar em largura, cresce muito rapido

**DFS**:O custo é menor pois só prescisamos manter a pilha dos nós do caminho que está sendo processado.

2.

**BFS**:É completo, consegue chegar a solução sem otimizações além de achar a solução mais "rasa", porem pode demorar muito, e tem a possibilidade estourar a memória.

**DFS**: Pode chegar rapidamente a solução quando está estiver em algum nivel mais profundo do que a busca BFS, mas não tem garantia que seja a mais "rasa", e sem alguma detecção de loop não é completo, também em casos como este em que a arvore de soluções pode ser infinita pode ser interessante limitar a profundidade

**Me parace que nenhum dos algoritmos é ideal para resolver este problema.**